In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_columns', 60)

In [7]:
movies = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

In [8]:
movies.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [9]:
movies.shape

(250, 38)

In [10]:
movies=movies[['Title','Genre','Director','Actors','Plot']]
movies.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [11]:

movies.isnull().sum()

Title       0
Genre       0
Director    0
Actors      0
Plot        0
dtype: int64

In [12]:
movies['Actors'] = movies['Actors'].map(lambda x: x.split(',')[:3])
# discarding the commas between the actors' full names and getting only the first three names


In [13]:
movies['Actors'] 

0            [Tim Robbins,  Morgan Freeman,  Bob Gunton]
1               [Marlon Brando,  Al Pacino,  James Caan]
2             [Al Pacino,  Robert Duvall,  Diane Keaton]
3        [Christian Bale,  Heath Ledger,  Aaron Eckhart]
4           [Martin Balsam,  John Fiedler,  Lee J. Cobb]
                             ...                        
245           [Ray Milland,  Jane Wyman,  Phillip Terry]
246    [Brie Larson,  John Gallagher Jr.,  Stephanie ...
247      [Cary Grant,  Rosalind Russell,  Ralph Bellamy]
248    [Sissy Spacek,  Jane Galloway Heitz,  Joseph A...
249           [Dev Patel,  Saurabh Shukla,  Anil Kapoor]
Name: Actors, Length: 250, dtype: object

In [14]:
movies['Genre']= movies['Genre'].map(lambda x: x.lower().split(','))
movies['Genre']

0                 [crime,  drama]
1                 [crime,  drama]
2                 [crime,  drama]
3        [action,  crime,  drama]
4                 [crime,  drama]
                  ...            
245           [drama,  film-noir]
246                       [drama]
247    [comedy,  drama,  romance]
248           [biography,  drama]
249                       [drama]
Name: Genre, Length: 250, dtype: object

In [15]:
movies['Director'] = movies['Director'].map(lambda x:x.split(',') )


In [16]:
movies['Director']

0                     [Frank Darabont]
1               [Francis Ford Coppola]
2               [Francis Ford Coppola]
3                  [Christopher Nolan]
4                       [Sidney Lumet]
                    ...               
245                     [Billy Wilder]
246            [Destin Daniel Cretton]
247                     [Howard Hawks]
248                      [David Lynch]
249    [Danny Boyle,  Loveleen Tandan]
Name: Director, Length: 250, dtype: object

In [17]:
movies['Director']

0                     [Frank Darabont]
1               [Francis Ford Coppola]
2               [Francis Ford Coppola]
3                  [Christopher Nolan]
4                       [Sidney Lumet]
                    ...               
245                     [Billy Wilder]
246            [Destin Daniel Cretton]
247                     [Howard Hawks]
248                      [David Lynch]
249    [Danny Boyle,  Loveleen Tandan]
Name: Director, Length: 250, dtype: object

In [18]:
for index, row in movies.iterrows():
    movies.at[index,'Actors']=[x.lower().replace(' ','') for x in row['Actors']]

In [19]:
movies['Actors']

0                 [timrobbins, morganfreeman, bobgunton]
1                    [marlonbrando, alpacino, jamescaan]
2                  [alpacino, robertduvall, dianekeaton]
3             [christianbale, heathledger, aaroneckhart]
4                 [martinbalsam, johnfiedler, leej.cobb]
                             ...                        
245                [raymilland, janewyman, phillipterry]
246     [brielarson, johngallagherjr., stephaniebeatriz]
247           [carygrant, rosalindrussell, ralphbellamy]
248    [sissyspacek, janegallowayheitz, josepha.carpe...
249                [devpatel, saurabhshukla, anilkapoor]
Name: Actors, Length: 250, dtype: object

In [20]:
for index, row in movies.iterrows():
    movies.at[index,'Director']=[x.lower().replace(' ','') for x in row['Director']]

In [267]:
movies['Director']

0                   [frankdarabont]
1              [francisfordcoppola]
2              [francisfordcoppola]
3                [christophernolan]
4                     [sidneylumet]
                   ...             
245                   [billywilder]
246           [destindanielcretton]
247                   [howardhawks]
248                    [davidlynch]
249    [dannyboyle, loveleentandan]
Name: Director, Length: 250, dtype: object

In [268]:
from rake_nltk import Rake


In [269]:
pip list

Package                       Version
----------------------------- ---------------
alabaster                     0.7.12
anaconda-client               1.11.2
anaconda-navigator            2.4.0
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.3
astroid                       2.14.2
astropy                       5.1
asttokens                     2.0.5
atomicwrites                  1.4.0
attrs                         22.1.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.11.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
black                         22.6.0
bleac

In [270]:
pip install rake_nltk

Note: you may need to restart the kernel to use updated packages.


In [271]:
import nltk

In [272]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mdipa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mdipa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [273]:
# initializing the new column
movies['Key_words'] = ""

for index, row in movies.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all punctuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
movies.drop(columns = ['Plot'], inplace = True)

In [274]:
key_words_dict_scores


defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [275]:
movies.set_index('Title', inplace = True)
movies.head()


,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",[frankdarabont],"[timrobbins, morganfreeman, bobgunton]","[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",[francisfordcoppola],"[marlonbrando, alpacino, jamescaan]","[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",[francisfordcoppola],"[alpacino, robertduvall, dianekeaton]","[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",[christophernolan],"[christianbale, heathledger, aaroneckhart]","[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",[sidneylumet],"[martinbalsam, johnfiedler, leej.cobb]","[jury, holdout, attempts, prevent, miscarriage..."


In [276]:
def create_bag_of_words(row):
    text = f"{row['Genre']} {row['Director']} {' '.join(row['Actors'])}"
    vectorizer = CountVectorizer()
    bag_of_words = vectorizer.fit_transform([text])
    return ' '.join(vectorizer.get_feature_names_out())

# Applying the function to create the 'bag_of_words' column
movies['bag_of_words'] = movies.apply(create_bag_of_words, axis=1)

# Dropping unnecessary columns
movies.drop(columns=['Genre', 'Director', 'Actors','Key_words'], inplace=True)

In [277]:
movies.head()

,bag_of_words
Title,
The Shawshank Redemption,bobgunton crime drama frankdarabont morganfree...
The Godfather,alpacino crime drama francisfordcoppola jamesc...
The Godfather: Part II,alpacino crime dianekeaton drama francisfordco...
The Dark Knight,aaroneckhart action christianbale christophern...
12 Angry Men,cobb crime drama johnfiedler leej martinbalsam...


In [280]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(movies['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(movies.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

In [281]:
count_matrix 


<250x765 sparse matrix of type '<class 'numpy.int64'>'
	with 1715 stored elements in Compressed Sparse Row format>

In [282]:
c=count_matrix.todense()


In [284]:
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [285]:
print(count_matrix[0,:])


  (0, 72)	1
  (0, 139)	1
  (0, 186)	1
  (0, 235)	1
  (0, 517)	1
  (0, 703)	1


In [286]:
count.vocabulary_


{'bobgunton': 72,
 'crime': 139,
 'drama': 186,
 'frankdarabont': 235,
 'morganfreeman': 517,
 'timrobbins': 703,
 'alpacino': 24,
 'francisfordcoppola': 232,
 'jamescaan': 319,
 'marlonbrando': 477,
 'dianekeaton': 176,
 'robertduvall': 610,
 'aaroneckhart': 0,
 'action': 3,
 'christianbale': 117,
 'christophernolan': 121,
 'heathledger': 290,
 'cobb': 131,
 'johnfiedler': 370,
 'leej': 439,
 'martinbalsam': 478,
 'sidneylumet': 666,
 'benkingsley': 59,
 'biography': 69,
 'history': 298,
 'liamneeson': 448,
 'ralphfiennes': 579,
 'stevenspielberg': 685,
 'adventure': 8,
 'aliastin': 21,
 'fantasy': 225,
 'noelappleby': 537,
 'peterjackson': 563,
 'seanastin': 655,
 'amandaplummer': 25,
 'lauralovelace': 432,
 'quentintarantino': 576,
 'timroth': 704,
 'bradpitt': 76,
 'davidfincher': 153,
 'edwardnorton': 202,
 'meatloaf': 493,
 'alanhoward': 10,
 'comedy': 134,
 'rebeccawilliams': 586,
 'robertzemeckis': 621,
 'romance': 627,
 'sallyfield': 642,
 'tomhanks': 707,
 'carriefisher': 96,

In [287]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.33333333, 0.33333333, ..., 0.15430335, 0.15430335,
        0.16666667],
       [0.33333333, 1.        , 0.66666667, ..., 0.15430335, 0.15430335,
        0.16666667],
       [0.33333333, 0.66666667, 1.        , ..., 0.15430335, 0.15430335,
        0.16666667],
       ...,
       [0.15430335, 0.15430335, 0.15430335, ..., 1.        , 0.14285714,
        0.15430335],
       [0.15430335, 0.15430335, 0.15430335, ..., 0.14285714, 1.        ,
        0.15430335],
       [0.16666667, 0.16666667, 0.16666667, ..., 0.15430335, 0.15430335,
        1.        ]])

In [292]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(movies.index)[i])
        
    return recommended_movies

In [293]:
recommendations('The Dark Knight')


[89, 51, 96, 40, 0, 83, 71, 68, 1, 54]


['Batman Begins',
 'The Dark Knight Rises',
 'Heat',
 'The Prestige',
 'The Shawshank Redemption',
 'Scarface',
 'Taxi Driver',
 'To Kill a Mockingbird',
 'The Godfather',
 'Once Upon a Time in America']

In [295]:
indices[indices == 'The Dark Knight'].index[0]


3